# Getting Started With Geoocoding

In this notebook, you will learn how to geocode different sorts of location data by making requests to several online APIs (*Application Proramming Interface*) for latitude and longitude co-ordinates associated with those locations.

The location data we will consider includes:

- <a href="#postcodes">postcodes</a>
- <a href="#addresses">addresses</a>
- <a href="#IPaddresses">IP addresses</a>
- <a href="#wifiHotspits">wifi access points</a>

In [ ]:
#The requests library makes it easy to call URLs using Python
import requests

<a name="postcodes"></a>
## Postcodes

Postcodes are a widely used form of location data, typically capable of identifying a location to a resolution of a few hundred square metres.

There are several online services that will return geolocation information given a postcode.

To call the service, we construct a URL as defined for a particular API and make a request to that URL using the *python* `requests` package.

Data is often returned from webservices using the JSON (Javascript Object Notation) data format, although some APIs allow you to specify other formats such as XML.

(One advantage of the JSON response is that it can be immediately consumed by a Javascript script called from inside a webpage.)

The *python* `requests` library has a method that parses a correctly formed JSON response as a *python* `dict`.

Run the following cell to call the `postcodes.io` API with a particular postcode.

See if you can make sense of the result that is returned.

In [ ]:
postcode = 'MK7 6AA'
r=requests.get('https://api.postcodes.io/postcodes/{PC}'.format(PC=postcode))
r.json()

Try rerunning the previous cell using different postcodes - can the service locate your home postcode?

### Parsing the `postcodes.io` JSON data

Once we have retrieved the data from the API, and cast it as a *python* data object, we can look inside it programmatically.

For example, we can find the latitude and longitude values.

In [ ]:
#Obtain the lat/long of a postcode
lat=r.json()['result']['latitude']
lon=r.json()['result']['longitude']

#Display the result
print(lat,lon)

Having access to the latitude and longitude means we can start to make use of that information, for example by plotting it on a map.

You may recall how we previously used the `folium` package to generate interactive maps from *python* code within a notebook.

We can do a similar thing again here.

In [ ]:
#Plot the lat long of a postcode on a map

#We need to import the following packages to access the maps
import folium

#Create a map centered on the postcode location at a particular zoom level
mymap = folium.Map(location=[lat, lon], zoom_start=15)

#Create a popup message using Python string formatting to create the label based on variable values
popupstr = 'Location of {PC}: ({lat},{lon})'.format(PC=postcode, lat=lat,lon=lon)

#Display a marker for the location
folium.Marker([52.0239, -0.7072], popup=popupstr).add_to(mymap)
mymap

<a name="addresses"></a>
## Addresses

As well as geolocating postcodes, we can also goecode complete (or partial) addresses. One API that supports address based geocoding is the Google Maps geocoding API.

Once again, we need to construct a URL according to a pattern defined by the API documentation. Then we can make a request to that URL and hopefully get the geocoded data back as a response.

In [ ]:
address='Open University, Walton Hall, Milton Keynes, MK7 6AA, UK'
r= requests.get("https://maps.googleapis.com/maps/api/geocode/json", params={'address': address, 'sensor': "false"})
r.json()

Try rerunning the previous cell with an address that is familiar to you. Does the API find it?

### Optional Activities

- see if you can write a loop that will look up the geolocations of several postcodes, one at a time. To be nice to the API import the *python* `time` library and add the statement `time.sleep(1)` inside the loop to pause its execution for one second during each iteration.
- create a new `folium` map object to display several markers, one for each of your (looped) postcodes. Inside the postcode loop add a corresponding marker to the map. Don't forget to render the map from the last line of code in the cell.

<a name="IPaddresses"></a>
## IP Addresses

As well as looking up geolocation data for a *postal* address, we can also try to look up a location based on the IP address of a computer. There are seveal websites that allow you to lookup the IP address of the device you are using to connect to the internet, and several webservices too.

I'm going to use a simple service from Amazon web services that returns an IP address terminated by an end of line (`\n`) character. By using the `requests` library, I can call the URL, access the data response (`text`) and then strip (`.strip()`)) the end-of-line whitespace character from it.

In [ ]:
myIPaddress=requests.get('http://checkip.amazonaws.com/').text.strip()
myIPaddress

In [ ]:
#We can construct a URL based around the IP address of the machine making the request as follows:
url='https://freegeoip.net/json/{IP}'.format(IP=myIPaddress)
url

In [ ]:
r=requests.get(url)
r.json()

The result may surprise you, for example if the notebook and the python process associated with it is running on a server hosted in the cloud. In this case, try looking up the IP address associated with computer you are using to access the internet by visiting the link: [http://checkip.amazonaws.com/](http://checkip.amazonaws.com/).


<a name="wifiHotspots"></a>
## WiFi Hotspot MAC Addresses

As well as services that provide access to directories that try to associate IP addresses with physical locations, there are also databases that also try to associate MAC addresses of wifi routers with physical locations.

If your computer has a wifi enabled, you will use access a low level command on your computer that identifies in-range wifi routers and provides adminstrative information about them.

**STILL NEEDS TESTING & REFINING - TO DO** 

Note that to call the Google webservice used in this example that associates router MAC address with locations, you will need to get a Google Geolocation API token: visit [https://developers.google.com/maps/documentation/geocoding/get-api-key](https://developers.google.com/maps/documentation/geocoding/get-api-key) and follow the instructions on how to get a key for the geolocation API.

When you have obtained your key, use it to set the `googleMapsAPIkey` variable below.

Also note that the code may look a little bit involved. But `DON'T PANIC`, you don't need to be able to write, or even read, this sort of code for the purposes of this course.

In [ ]:
googleMapsAPIkey="YOUR_KEY_HERE"

In [ ]:
import sys

#http://stackoverflow.com/a/9859202/454773
def isInt_str(v):
    v = str(v).strip()
    return v=='0' or (v if v.find('..') > -1 else v.lstrip('-+').rstrip('0').rstrip('.')).isdigit()


#/System/Library/PrivateFrameworks/Apple80211.framework/Resources/airport
import subprocess
def getWifiMacAddresses():
    #autodetect platform and then report based on this? 
    print(sys.platform)
    
    macAddr={}
    
    #For Mac:
    if sys.platform=='darwin':
            results = subprocess.check_output(["/System/Library/PrivateFrameworks/Apple80211.framework/Resources/airport", "-s"])
            results = results.decode("utf-8") # needed in python 3
            ls = results.split("\n")
            ls = ls[1:]
            for l in [x.strip() for x in ls if x.strip()!='']:
                ll=l.split(' ')
                #We could use a regular expression - or we can cosntruct our parser a step at a time...
                macAddr[l.strip().split(' ')[0]]=(l.strip().split(' ')[1], l.strip().split(' ')[2])
                
    elif sys.platform=='??windows':
        #Need a windows machine to explore this - TO DO
        #win?
        #results = subprocess.check_output(["netsh", "wlan", "show", "network"])

        results = subprocess.check_output(["netsh", "wlan", "show", "network"])

        #Adding: / mode=bssid / gives SSIDS?#or maybe: netsh wlan show allresults = results.decode("ascii") # needed in python 3
        results = results.replace("\r","")
        ls = results.split("\n")
        ls = ls[4:]
        ssids = []
        x = 0
        while x < len(ls):
            if x % 5 == 0:
                ssids.append(ls[x])
            x += 1
        print(ssids)

    elif sys.platform=='??linux':
        #linux?
        #! apt-get -y install wireless-tools
        #results = subprocess.check_output(["iwlist","scanning"])    
        #via PP - linux text - TO DO
        # apt-get -y install wireless-tools then run iwlist scanning to display the details of wireless access points your computer can see.
        #apt-get -y install wireless-tools gave me "Could not open the lock file ..."
        #However when I checked in the Ubuntu Software Centre wireless-tools was already installed. I think non-expert users may use the Software Centre to install additional applications.
        #iwlist just give you a not very helpful usage list. What works directly is:
        #iwlist wlan0 scan
        pass

    return macAddr

postjson={'wifiAccessPoints':[]}
hotspots=getWifiMacAddresses()
for h in hotspots:
    addr,db=hotspots[h]
    if isInt_str(db):
        postjson['wifiAccessPoints'].append({'macAddress':addr, 'signalStrength':int(db)})
        print('{}: {}'.format(h,hotspots[h]))

url='https://www.googleapis.com/geolocation/v1/geolocate?key={}'.format(googleMapsAPIkey)

r = requests.post(url, json=postjson)
r.json()

## Summary

In this notebook, you have learned how to geocode several different sorts of location identifer - postcodes, postal addresses, IP addresses and maybe even the MAC address of any WiFI routers in view of your computer.

You have also seen how we can take the JSON data returned from the geolocation services and parse it as python dict that we can then start to work *as data*, for example, by plotting markers associated with identified locations on an interactive map.